In [24]:
import numpy as np
import pandas as pd
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [31]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-08-31',today).loc[:,['date','close']]
bndxPrice = getPrice(token,'bndx','2019-08-31',today).loc[:,['date','close']]
vtiPrice = getPrice(token,'vti','2019-08-31',today).loc[:,['date','close']]
vxusPrice = getPrice(token,'vxus','2019-08-31',today).loc[:,['date','close']]

In [32]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_Data.csv')
vti['year'] = pd.DatetimeIndex(vti['date']).year
vti['month'] = pd.DatetimeIndex(vti['date']).month
vxus = pd.read_csv('Data/vxus_MC_Data.csv')
vxus['year'] = pd.DatetimeIndex(vxus['date']).year
vxus['month'] = pd.DatetimeIndex(vxus['date']).month
bnd = pd.read_csv('Data/bnd_MC_Data.csv')
bnd['year'] = pd.DatetimeIndex(bnd['date']).year
bnd['month'] = pd.DatetimeIndex(bnd['date']).month
bndx = pd.read_csv('Data/bndx_MC_Data.csv')
bndx['year'] = pd.DatetimeIndex(bndx['date']).year
bndx['month'] = pd.DatetimeIndex(bndx['date']).month

In [33]:
df = vtiPrice.rename(columns={"close":"VTI_ClosePrice"}).merge(vxusPrice.rename(columns={"close":"VXUS_ClosePrice"}),on='date',how='outer').merge(bndPrice.rename(columns={"close":"BND_ClosePrice"}),on='date',how='outer').merge(bndxPrice.rename(columns={"close":"BNDX_ClosePrice"}),on='date',how='outer')
df

,date,VTI_ClosePrice,VXUS_ClosePrice,BND_ClosePrice,BNDX_ClosePrice
0,2019-09-03,148.07,50.38,84.99,59.220
1,2019-09-04,149.65,51.14,85.15,59.030
2,2019-09-05,151.65,51.53,84.71,58.810
3,2019-09-06,151.68,51.69,84.81,58.880
4,2019-09-09,151.77,51.75,84.42,58.730
...,...,...,...,...,...
223,2020-07-22,165.77,52.20,89.11,58.080
224,2020-07-23,163.96,51.80,89.19,58.070
225,2020-07-24,162.69,51.61,89.17,58.040
226,2020-07-27,164.09,52.35,89.11,58.115


In [16]:
vtiPrice

,date,close
0,2016-07-01,84.37
1,2016-07-05,84.66
2,2016-07-06,84.81
3,2016-07-07,84.67
4,2016-07-08,84.89
...,...,...
876,2019-12-24,83.78
877,2019-12-26,83.86
878,2019-12-27,83.98
879,2019-12-30,83.95


In [13]:
# Enter in total amount you would like to invest under amountInvested 
amountInvested = 12000

In [14]:
# GENERATING MARKET PORTFOLIO
# format [U.S. Stocks, International Stocks, U.S. Bonds, International Bonds] (in billions for market cap)
EOQmarketcaps = np.array(etfEOQmarketcaps)
EOQprices = np.array(etfEOQprices)
recentPrices = np.asarray(etfRecentPrices)
# Market proportions for each etf on end of quarter
EOQproportions = EOQmarketcaps/sum(EOQmarketcaps)
# Ratio of recent prices and end of quarter prices
priceRatios = recentPrices / EOQprices
# Multiply ratios to market proportions
products = EOQproportions * priceRatios
# Market weights if fractional shares existed
revisedProportions = products / sum(products)
# Amount of money invested in each etf if fractional shares existed
desiredValues = revisedProportions * amountInvested
# Number of shares of etfs to buy if fractional shares existed
currentPrices = np.array(etfCurrPrices(['VTI', 'VXUS', 'BND', 'BNDX']))
shares = desiredValues/currentPrices
# Optimize to get as close to 0 since fractional shares with Schwab doesn't exist
def asdf(a):
    return amountInvested - sum(np.round(shares * a)*currentPrices)
a = .90
while asdf(a) > 0:
    if a <.99999:
        a+=.00001
    else:
        break
    if asdf(a) < 0:
        a-=.00001
        break
# Actual shares of each etf to buy
actualShares = np.round(shares * a)
print('VTI, VXUS, BND, BNDX')
print(actualShares)

VTI, VXUS, BND, BNDX
[26. 63. 34. 41.]


In [15]:
actualProportions = (actualShares * currentPrices)/sum(actualShares * currentPrices)
print('Theoretical proportions')
print(revisedProportions)
print('Actual proportions')
print(actualProportions)

Theoretical proportions
[0.32022161 0.23760532 0.24689227 0.19528079]
Actual proportions
[0.31855103 0.23872327 0.24735598 0.19536972]


In [16]:
# Cash left over. You can invest this cash into VTIP (Vanguard's US TIPS ETF) or something else if you'd like.
cash = amountInvested - sum((actualShares * currentPrices))
print(np.round(cash,2))

0.29


In [17]:
# How much it costs to maintain portfolio per year
costs = [.0003,.0008,.00035,.0008]
print(sum((actualShares * currentPrices)*costs))

6.352818000000001


In [12]:
currentPrices

array([147.02,  45.47,  87.3 ,  57.18])

In [ ]:
# REBALANCE (run this cell if you already have money invested in the market portfolio)
# Enter in number of shares here in current port in the format of VTI, VXUS, BND, BNDX
currentPort = np.array([12.0588,27.0695,16.0667,21.0345])
trades = actualShares - currentPort
# Amount left for trades
amountInvestable = amountInvested - sum(currentPort * currentPrices)
# Need to make sure trades don't make balance go over amount of money available
def asdf(a):
    return amountInvestable - sum(np.round(trades * a)*currentPrices)
a = .90
while asdf(a) > 0:
    if a <.99999:
        a+=.00001
    else:
        break
    if asdf(a) < 0:
        a-=.00001
        break
actualTrades = np.round(trades * a)
print('Trades: VTI, VXUS, BND, BNDX')
print(actualTrades)
print()
print('Cash left over')
print(asdf(a))

In [ ]:
portAfterTrades = currentPort + actualTrades
print('Theoretical proportions')
print(revisedProportions)
print()
print('Actual proportions after rebalance')
print((portAfterTrades * currentPrices)/sum(portAfterTrades * currentPrices))